In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; }
.container {width:95% !important;}
</style>

<h1 style="text-align:center">Clase: Analicemos el algoritmo de la transformada rápida de Fourier (FFT)</h1>
<h3 style="text-align:center">ACUS099: Procesamiento digital de señales</h3>
<p style="text-align:center">Dr. Víctor Poblete <br>
<a href="mailto:vpoblete@uach.cl">vpoblete@uach.cl</a><br>
<a href="https://github.com/vpobleteacustica/ACUS099-Procesamiento-digital-de-senales">https://github.com/vpobleteacustica/ACUS099-Procesamiento-digital-de-senales</a><br> 
<p style="text-align:center">Diego Espejo Alquinta - Ayudante <br>
<a href="mailto:diego.espejo@alumnos.uach.cl">diego.espejo@alumnos.uach.cl </a><br>
<a href="http://www.acusticauach.cl">www.acusticauach.cl</a><br> 

In [1]:
from scipy.fftpack import fft, fftshift, ifft
from scipy.fftpack import fftfreq
import scipy.io.wavfile as wavfile
import scipy
from scipy import signal
import librosa
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import EngFormatter
%matplotlib notebook


### La transformada continua de Fourier (FT)
> + La transformada continua de Fourier se usa para representar una **función** continua como una suma de armónicos constituyentes. 
> + Es una transformación **lineal**, e **invertible**, entre la representación en el dominio del **tiempo** de una función, que denotaremos por $x(t)$, y el dominio de la **frecuencia**,
representación que denotaremos por $X(\omega)$. 
> + En una dimensión como es el caso de señales de audio, el **par de transformadas de Fourier** que consiste en la **transformada directa** y
**transformada inversa**, se definen como:
$$
\begin{align*}
X(\omega) = & \int_{-\infty}^{\infty} x(t)\cdot e^{-j\omega t} dt\quad \quad\quad{\text{(transformada directa)}}\\
x(t)      = & \,\frac{1}{2\pi}\int_{-\infty}^{\infty} X(\omega)\cdot e^{j\omega t} d\omega \quad \,{\text{(transformada inversa)}}
\end{align*}
$$
> + donde $j=\sqrt{-1}$, y la exponencial compleja: $e^{j\omega t}= \cos\omega t + j\sin\omega t$.
> + Se puede pensar que es una **transformación en un conjunto diferente de funciones base**. 
> + Como funciones base se utilizan **exponenciales complejas** de diversas frecuencias. 
> + Existen otras transformadas, como Z, Laplace, Coseno, Wavelet, pero se diferencian porque usan otras funciones base.
> + Un par de transformadas de Fourier a menudo se escribe como: 
>> + $x(t)\overset{\mathscr{F}}{\longleftrightarrow} X(\omega)$
>> + O bien, como $\mathscr{F}\left\lbrace x(t) \right\rbrace = X(\omega)$
>> + donde $\mathscr{F}\left\lbrace \cdot \right\rbrace$ es el operador matemático transformada de Fourier que **opera** sobre la función continua.
> + Notar que la transformada de Fourier se define naturalmente en términos de funciones complejas. 
> + Tanto $x(t)$ como $X(\omega)$ son, en general, funciones de valor complejo.
> + Si $x(t)$ es nuestra señal de audio (es decir, nuestros datos de entrada son las amplitudes de una señal de audio), nos referimos a $F(\omega)$ como el **espectro de la señal** (con un número continuo de frecuencias).


### La transformada discreta de Fourier (DFT)
> + Si trabajamos sobre un computador, usamos la **Transformada Discreta de Fourier** (DFT). 
> + Las ecuaciones para la DFT son muy similares a las de la transformada continua de Fourier.
> + En este caso, tanto la **función** como su **transformada de Fourier** se reemplazan por sus contrapartes **discretas** (conjunto discreto de muestras), y se denominará la DFT.
> + Por lo tanto, la DFT trabaja con señales de **duración finita**, de duración $N$.
> + Análogamente, el **par de transformadas discretas de Fourier** que consiste en la **transformada discreta directa** y
**transformada discreta inversa**, se definen como:
$$
\begin{align*}
X[k] = & \sum_{n=0}^{N-1} x[n] \cdot W_N^{kn} \quad \quad\quad{\text{(transformada discreta directa)}}\\
x[n]      = & \,\frac{1}{N}\sum_{n=0}^{N-1} X[k]\cdot W_N^{-kn} \quad \,{\text{(transformada discreta inversa)}}
\end{align*}
$$
> + donde $x[n]$ es una señal de audio en tiempo discreto y de duración $N$, $X[k]$ es el **espectro discreto de la señal** (con un número discreto de frecuencias), y $W_N=e^{-j\frac{2\pi}{N}}$.

### La trasformada rápida de Fourier (FFT)
> + La FFT es un algoritmo rápido para calcular la DFT de una secuencia discreta.
> + Fue publicado por primera vez por **Cooley** (IBM) y **Tukey** (Princeton University) en 1965 [https://www.ams.org/journals/mcom/1965-19-090/S0025-5718-1965-0178586-1/S0025-5718-1965-0178586-1.pdf].
> + Sin embargo, la idea original se remonta a un artículo inédito de Gauss en 1805 [https://www.cis.rit.edu/class/simg716/Gauss_History_FFT.pdf]. 
> + La FFT es considerada como uno de los algoritmos más importantes en computación científica.
> + El algoritmo de la FFT va dividiendo recursivamente la DFT, en DFTs más pequeños para acelerar el cálculo. 
> + Como resultado el tiempo de cómputo se reduce asombrosamente, ya que la cantidad de operaciones de la DFT original que es de un orden $N^{2}$, disminuye a un orden de ($N\cdot {\text{log}}_2{N}$) operaciones, donde $N$ es el número total de datos. 
> + Esta reducción en el tiempo de cómputo es muy importante, especialmente para conjuntos con grandes volúmenes de datos. 
> + La FFT es más eficiente si su **largo N** (se dice que $N$ es el largo de la FFT) es una potencia de dos: $N = 2^{L} \,\, {\text{elementos}}.$
> + Basada en el lema de Danielson-Lanczos [https://mathworld.wolfram.com/Danielson-LanczosLemma.html], la FFT de largo $N$ se escribe como una suma de 2 DFTs de largo $N/2$.
> + La FFT la podemos aplicar al largo entero de la señal en el tiempo discreto, por lo que el largo de la FFT sería igual al largo de la señal. Sin embargo, ya que la FFT es más eficiente cuando su largo es una potencia de 2, se aplica una técnica muy clásica llamada **zero padding** que agrega ceros hasta que la señal llega a ser de un largo que es potencia de 2.
>> + Por ejemplo, supongamos que $x[n]$ tiene largo 3: $x = [3.4, 2.56, 1.3]$, de tal manera que cambiamos $x[n]$ para que sea: $x = [3.4, 2.56, 1.3, 0.0]$, y aplicamos una FFT de largo 4.
> + Un gran PERO... es que si nuestra señal es muy larga, se vuelve extremadamente ineficaz hacer todo a la vez. No es práctico intentar hacer una FFT a un archivo de audio de la larga longitud. En ese caso, dividimos la señal en ventanas de un tamaño razonable, y realizamos una FFT en cada una de esas ventanas y promediamos los resultados.
> + La DFT nos proporciona **información sobre un número discreto de frecuencias**, por lo que debemos determinar qué frecuencias son éstas.
> + Para esto, muestreamos una señal continua $x(t)$ a intervalos equidistantes espaciados $\Delta t$, generando una secuencia de valores muestreados.
> + El recíproco de este espaciamiento $\Delta t$, nos da la **tasa de muestreo**, o también llamada **frecuencia de muestreo:** 
$$
\begin{align*}
f_s &= \frac{1}{\Delta t}\quad \quad\quad{\text{(frecuencia de muestreo)}}
\end{align*}
$$
> + Para cualquier intervalo de muestreo $\Delta t$, existe la frecuencia $f_{max} $ llamada **frecuencia límite de Nyquist**:
$$
\begin{align*}
f_{max} &= \frac{f_s}{2}\quad \quad\quad{\text{(frecuencia de Nyquist)}}
\end{align*}
$$
> + La frecuencia límite de Nyquist es la frecuencia más alta que puede ser representada por el proceso de muestreo a intervalos de $\Delta t$. 
> + Debemos muestrear la señal $x(t)$ al menos 2 veces la frecuencia más alta contenida en la señal.
> + Si $f_s =$ 1KHz, la frecuencia límite de Nyquist es 500 Hz.


